In [1]:
import pytensor
import pytensor.tensor as pt
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def flatten_equations(eqs):
    return pt.concatenate([pt.atleast_1d(eq).ravel() for eq in eqs], axis=-1)


def make_jacobian(system, x, n_eq):
    column_list = pytensor.gradient.jacobian(system, x)
    jac = pt.concatenate([pt.atleast_2d(x).reshape((n_eq, -1)) for x in column_list], axis=-1)

    return jac

In [73]:
variables = v1, v2 = [pt.dscalar(name) for name in ["v1", "v2"]]
parameters = v3 = pt.dscalar("v3")
inputs = variables + parameters

equations = [v1**2 * v3 - 1, v1 + v2 - 2]

In [78]:
pt.atleast_1d(v3).ravel()

SpecifyShape.0

In [79]:
def at_least_list(x):
    if isinstance(x, list):
        return x
    else:
        return [x]


def clone_and_rename(x, suffix="_next"):
    x_new = x.clone()
    x_new.name = f"{x.name}{suffix}"
    return x_new


def euler_approximation(system, variables, parameters):
    n_eq = len(system)
    n_steps = pt.iscalar("n_steps")
    flat_system = flatten_equations(system)

    x_list = at_least_list(variables)
    theta_list = at_least_list(parameters)
    theta_final = pt.stack([clone_and_rename(x) for x in theta_list])

    x0 = flatten_equations(x_list)
    theta0 = flatten_equations(theta_list)

    dtheta = theta_final - theta0
    step_size = dtheta / n_steps

    A = make_jacobian(flat_system, x_list, n_eq)
    A_inv = pt.linalg.solve(A, pt.identity_like(A), check_finite=False)
    A_inv.name = "A_inv"

    B = make_jacobian(flat_system, theta_list, n_eq)
    Bv = B @ pt.atleast_1d(step_size)
    Bv.name = "Bv"

    step = -A_inv @ Bv
    f_step = pytensor.compile.builders.OpFromGraph(
        x_list + theta_list + [step_size], [step], inline=True
    )

    def step_func(x_prev, theta_prev, flat_system, theta_final, n_steps):
        step = f_step(*x_prev, *theta_prev, step_size)

        x = x_prev + step
        theta = theta_prev + step_size

        return x, theta

    result, updates = pytensor.scan(
        step_func,
        outputs_info=[pt.stack(x_list), pt.stack(theta_list)],
        non_sequences=[flat_system, theta_final, n_steps],
        n_steps=n_steps,
    )

    return [theta_final, n_steps], result

In [100]:
new_inputs, result = euler_approximation(equations, variables, parameters)
theta_final, n_steps = new_inputs

In [101]:
inputs = [v1, v2, v3, n_steps, theta_final]
f = pytensor.function(inputs, result, mode="NUMBA")
f.vm.jit_fn(1.0, 1.0, 1.0, 10_000, np.array([2.0]))

/var/folders/7b/rzxy96cj0w751_6td3g2yss00000gn/T/tmpfttl9nn5:15: NumbaPendingDeprecationWarning: Code using Numba extension API maybe depending on 'old_style' error-capturing, which is deprecated and will be replaced by 'new_style' in a future release. See details at https://numba.readthedocs.io/en/latest/reference/deprecation.html#deprecation-of-old-style-numba-captured-errors
Exception origin:
  File "/Users/jessegrabowski/mambaforge/envs/cge-dev/lib/python3.11/site-packages/llvmlite/ir/instructions.py", line 105, in __init__
    raise TypeError(msg)

  tensor_variable_10 = elemwise_1(tensor_variable_9, tensor_variable_8, tensor_variable_6)


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mNo implementation of function Function(<function numba_funcify_Elemwise.<locals>.elemwise at 0x2a6acf4c0>) found for signature:
 
 >>> elemwise(readonly array(int64, 0d, C), int64, readonly array(int64, 0d, C), readonly array(int64, 0d, C), readonly array(int64, 0d, C), readonly array(int64, 0d, C), readonly array(int64, 0d, C), readonly array(int64, 0d, C), readonly array(int8, 0d, C), readonly array(int8, 0d, C), readonly array(int8, 0d, C), readonly array(int8, 0d, C), readonly array(int8, 0d, C), readonly array(int8, 0d, C), readonly array(int8, 0d, C), readonly array(int64, 0d, C), readonly array(int8, 0d, C), readonly array(int64, 0d, C), readonly array(int8, 0d, C), readonly array(int8, 0d, C), readonly array(int8, 0d, C), readonly array(int64, 0d, C), readonly array(int64, 0d, C))
 
There are 2 candidate implementations:
[1m  - Of which 2 did not match due to:
  Overload in function 'numba_func

{v1: [None],
 v2: [None],
 v3: [None],
 n_steps: [None],
 <Vector(float64, shape=(1,))>: [None],
 TensorConstant(TensorType(int64, shape=()), data=array(1)): [array(1)],
 TensorConstant(TensorType(int8, shape=()), data=array(1, dtype=int8)): [array(1, dtype=int8)],
 TensorConstant(TensorType(int8, shape=()), data=array(2, dtype=int8)): [array(2, dtype=int8)],
 Composite{...}.0: [None],
 Composite{...}.1: [None],
 Composite{...}.2: [None],
 Composite{...}.3: [None],
 Composite{...}.4: [None],
 Composite{...}.5: [None],
 ScalarFromTensor.0: [None],
 ScalarFromTensor.0: [None],
 ExpandDims{axis=0}.0: [None],
 TensorConstant(TensorType(int64, shape=(1,)), data=array([-1])): [array([-1])],
 Reshape{1}.0: [None],
 SpecifyShape.0: [None],
 Composite{((i0 - i1) / i2)}.0: [None],
 MakeVector{dtype='float64'}.0: [None],
 ExpandDims{axis=0}.0: [None],
 Unbroadcast{0}.0: [None],
 AllocEmpty{dtype='float64'}.0: [None],
 ScalarConstant(ScalarType(int64), data=1): [1],
 SetSubtensor{:stop}.0: [None],

In [98]:
x = pt.dscalar('x')
y = pt.dvector('y')
z = 

f = pytensor.function([x, y], z, mode='NUMBA', on_unused_input='ignore')

f.vm.jit_fn(1., np.ones(3))

(array([-0.41614684, -0.41614684, -0.41614684]),)

In [65]:
%timeit f(1., 1., 1., 10_000, np.array([2.]))

718 ms ± 161 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
